In [7]:
import os
import numpy as np
import pandas as pd
import folium as fl
from IPython.core.display import display, HTML
import json as js
%matplotlib inline

In [3]:
data_folder = 'data/'

# Part 1 
Displaying the percentage of voters for a party in each canton

In [5]:
#load data
geo_data = os.path.join('data', 'geojson.json')
geo_json_data = js.load(open(geo_data))
geo_json_data_2 =geo_json_data.copy()

# Remove Grison and Appenzell Innerrhoden
geo_json_data_2['features']=list(filter(lambda x: x['id'] != 'GR' and x['id'] != 'AI',geo_json_data['features']))
topo_json_data = js.load(open(data_folder + 'ch-cantons.topojson.json'))
#check number of cantons
len(geo_json_data_2['features'])

24

Displaying Switzerland on the map:

In [8]:
swiss_coord = [46.8131873 , 8.22421]
swiss = fl.Map(location=swiss_coord,tiles='cartodbpositron',
           zoom_start=8)
swiss.save('switzerland_map.html')
display(HTML("<h1><a href='switzerland_map.html' target='_blank'>Switzerland (Ctrl+Click)</a></h1>"))

swiss

Displaying the cantons on the map:

In [13]:
topo_data = os.path.join('data', 'ch-cantons.topojson.json')

fl.TopoJson(
    open(topo_data),
    'objects.cantons',
    name='topojson'
).add_to(swiss)

swiss.save('switzerland_cantons.html')
display(HTML("<h1><a href='switzerland_cantons.html' target='_blank'>Switzerland cantons (Ctrl+Click)</a></h1>"))

swiss

Displaying percentage of UDC voters per canton on the map

In [14]:
#Loading data 
voters = pd.read_excel(data_folder + "voters.xls", sheet_name='actuel (2014-2018)', index_col=0, skiprows=3)

#Removing NA values, fill others with 0 value
voters.dropna(how='all', inplace=True)
voters.dropna(how='all', axis= 1, inplace=True)
voters.fillna(0, inplace= True)

#Fetching UDC voters percentages per cantons
UDCvotersPercentage_perCanton = pd.DataFrame(voters['UDC'])
UDCvotersPercentage_perCanton.index.name='Cantons'

#Parsing string
UDCvotersPercentage_perCanton.set_index(UDCvotersPercentage_perCanton.index.str.replace('\d+','').str.replace(')', ''), inplace=True)
UDCvotersPercentage_perCanton = UDCvotersPercentage_perCanton[UDCvotersPercentage_perCanton["UDC"] != 0]

#Loading cantons data
cantons = pd.read_csv(data_folder + 'cantons.csv', index_col=1)
cantons.drop(['Grisons', 'Appenzell Innerrhoden'], inplace= True)

#Changing cantons name to cantons code
UDCvotersPercentage_perCanton['Code'] = cantons['Code'].values
UDCvotersPercentage_perCanton.reset_index(inplace=True)
UDCvotersPercentage_perCanton = UDCvotersPercentage_perCanton[['Code', 'UDC']]

print("Dataframe with percentage of UDC voters per canton:")
UDCvotersPercentage_perCanton

Dataframe with percentage of UDC voters per canton:


Code        UDC
0    ZH  30.023215
1    BE  26.760869
2    LU  24.115646
3    UR  24.053191
4    SZ  33.115090
5    OW  24.532303
6    NW  25.916631
7    GL  25.275361
8    ZG  23.631802
9    FR  19.717606
10   SO  19.734534
11   BS  14.297919
12   BL  26.741611
13   SH  35.072440
14   AR  16.281936
15   SG  29.549176
16   AG  31.937510
17   TG  32.555828
18   TI   4.286326
19   VD  15.842675
20   VS  16.437539
21   NE  11.489286
22   GE   7.315683
23   JU  11.980336

In [30]:
#Data
county_data = UDCvotersPercentage_perCanton
percentage_series = UDCvotersPercentage_perCanton.set_index('Code')['UDC']

results_percentage_series = fl.Map(location=[46.8, 8.33], zoom_start=7,tiles='cartodbpositron')
fl.TopoJson(
    open(topo_data),
    'objects.cantons',
    style_function=lambda x: {
        'fillOpacity': 1,
        'weight': 0,
        'fillColor': '#black'
    }
).add_to(results_percentage_series)

results_percentage_series.choropleth(geo_data=geo_json_data_2, 
    data=percentage_series,
    columns=['Code', 'UDC' ],
    key_on='feature.id',
    fill_color='GnBu', fill_opacity=0.9, line_opacity=0.2,
    legend_name='Percentage of UDC voters per cantons',
    reset=True)

results_percentage_series.save('Percent UDC voters.html')
display(HTML("<h1><a href='Percent UDC voters.html' target='_blank'>Percent UDC voters per Canton (Ctrl+Click)</a></h1>"))
results_percentage_series